In [1]:
import numpy as np
from PIL import Image, ImageDraw, ImageColor
import matplotlib.pyplot as plt

In [ ]:
image = np.array(Image.open('pops_512.jpg'))
display(Image.fromarray(image, mode='RGB'))

r, g, b = image[:, :, 0], image[:, :, 1], image[:, :, 2]

In [3]:
x, y = image.shape[1], image.shape[0]
e_x, e_y = 50, 50

bbox = (
    (x/2)-(e_x/2),
    (y/2)-(e_y/2),
    (x/2)+(e_x/2),
    (y/2)+(e_y/2),
)

low_pass = Image.new("L", (x, y), color=0)

low_pass_draw = ImageDraw.Draw(low_pass)
low_pass_draw.ellipse(bbox, fill=255)

low_pass_np = np.array(low_pass)

In [4]:
def normalise(arr, clamp_zero=False):
    min_val = np.min(arr) if not clamp_zero else 0
    max_val = np.max(arr)

    if min_val == max_val:
        return np.full_like(arr, 0, dtype=np.uint8)

    scale_factor = 255 / (max_val - min_val)
    norm_arr = (arr - min_val) * scale_factor

    return np.clip(np.round(norm_arr), 0, 255).astype(np.uint8)

In [5]:
F_r = np.fft.fftshift(np.fft.fft2(r))
F_g = np.fft.fftshift(np.fft.fft2(g))
F_b = np.fft.fftshift(np.fft.fft2(b))

F_r_filt = F_r * low_pass_np
F_g_filt = F_g * low_pass_np
F_b_filt = F_b * low_pass_np

r_filt = normalise(np.fft.ifft2(np.fft.ifftshift(F_r_filt)).real)
g_filt = normalise(np.fft.ifft2(np.fft.ifftshift(F_g_filt)).real)
b_filt = normalise(np.fft.ifft2(np.fft.ifftshift(F_b_filt)).real)

In [ ]:
fig, axs = plt.subplots(3, 2)

fig.patch.set_visible(False)

for ax in axs.ravel():
    ax.axis('off')

eps = 1.e-30

axs[0, 0].imshow(np.log(np.abs(F_r)), cmap='Reds_r')
axs[0, 1].imshow(np.log(np.abs(F_r_filt.real + eps)), cmap='Reds_r')
axs[1, 0].imshow(np.log(np.abs(F_g.real)), cmap='Greens_r')
axs[1, 1].imshow(np.log(np.abs(F_g_filt.real + eps)), cmap='Greens_r')
axs[2, 0].imshow(np.log(np.abs(F_b.real)), cmap='Blues_r')
axs[2, 1].imshow(np.log(np.abs(F_b_filt.real + eps)), cmap='Blues_r')

plt.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(3, 2)

fig.patch.set_visible(False)

for ax in axs.ravel():
    ax.axis('off')

axs[0, 0].imshow(r, cmap='Reds_r')
axs[0, 1].imshow(r_filt, cmap='Reds_r')
axs[1, 0].imshow(g, cmap='Greens_r')
axs[1, 1].imshow(g_filt, cmap='Greens_r')
axs[2, 0].imshow(b, cmap='Blues_r')
axs[2, 1].imshow(b_filt, cmap='Blues_r')

plt.tight_layout()
plt.show()

In [ ]:
fft_orig = normalise(np.log(np.abs(np.dstack((F_r, F_g, F_b)))), clamp_zero=True)
fft_filt = normalise(np.log(np.abs(np.dstack((F_r_filt, F_g_filt, F_b_filt)))), clamp_zero=True)
image_filt = normalise(np.dstack((r_filt, g_filt, b_filt)), clamp_zero=True)

display(Image.fromarray(fft_orig, mode='RGB'))
display(Image.fromarray(fft_filt, mode='RGB'))
display(Image.fromarray(image_filt, mode='RGB'))